<a href="https://colab.research.google.com/github/leejunho12316/NLP-study/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_12_IMDB_%EC%98%81%ED%99%94_%ED%8F%89%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from tensorflow.keras import datasets

In [47]:
#데이터
(x_train, y_train), (x_test, y_test) = datasets.imdb.load_data(num_words = 10000)

In [63]:
#데이터 디코딩 데이터
word_to_index = datasets.imdb.get_word_index()

word_to_index = {k:(v+3) for k,v in word_to_index.items()}
word_to_index["<PAD>"]=0
word_to_index["<START>"]=1
word_to_index["<UNK>"]=2
word_to_index["<UNUSED>"]=3

reverse_index = dict((val, key) for (key, val) in word_to_index.items())

changed = ' '.join([reverse_index[word] for word in x_train[0]])
changed

"cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all"

In [64]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(x_train, maxlen=100)
x_test = pad_sequences(x_test, maxlen=100)

In [65]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.Embedding(10000, 64, input_length = 100))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 64)           640000    
                                                                 
 flatten_1 (Flatten)         (None, 6400)              0         
                                                                 
 dense_2 (Dense)             (None, 64)                409664    
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1049729 (4.00 MB)
Trainable params: 1049729 (4.00 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [66]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=64, epochs=20, verbose=1, validation_data = (x_test, y_test))

Epoch 1/20
391/391 [==============================] - 11s 26ms/step - loss: 0.4637 - accuracy: 0.7629 - val_loss: 0.3377 - val_accuracy: 0.8481
Epoch 2/20
391/391 [==============================] - 10s 25ms/step - loss: 0.1666 - accuracy: 0.9409 - val_loss: 0.4023 - val_accuracy: 0.8347
Epoch 3/20
391/391 [==============================] - 11s 29ms/step - loss: 0.0236 - accuracy: 0.9953 - val_loss: 0.5455 - val_accuracy: 0.8337
Epoch 4/20
391/391 [==============================] - 10s 25ms/step - loss: 0.0046 - accuracy: 0.9997 - val_loss: 0.5994 - val_accuracy: 0.8383
Epoch 5/20
391/391 [==============================] - 9s 24ms/step - loss: 0.0016 - accuracy: 0.9999 - val_loss: 0.6572 - val_accuracy: 0.8375
Epoch 6/20
391/391 [==============================] - 10s 25ms/step - loss: 9.0620e-04 - accuracy: 1.0000 - val_loss: 0.6898 - val_accuracy: 0.8401
Epoch 7/20
391/391 [==============================] - 10s 25ms/step - loss: 5.3849e-04 - accuracy: 1.0000 - val_loss: 0.7199 - val_ac

In [67]:
results = model.evaluate(x_test, y_test, verbose=2)
print(results)

782/782 - 2s - loss: 1.0364 - accuracy: 0.8399 - 2s/epoch - 2ms/step
[1.036352276802063, 0.8399199843406677]


마지막 내가 쓴 리뷰로 평가 시켜보기

In [74]:
import re

review = "This is really really good f a great movie"
review = re.sub("[^0-9a-zA-Z ]", "", review).lower()

review_encoding = []
# 리뷰의 각 단어 대하여 반복한다.
for w in review.split():
		index = word_to_index.get(w, 2)	# 딕셔너리에 없으면 2 반환
		if index <= 10000:		# 단어의 개수는 10000이하
			review_encoding.append(index)
		else:
			review_encoding.append(word_to_index["<UNK>"])

# 2차원 리스트로 전달하여야 한다.
test_input = pad_sequences([review_encoding], maxlen = 100)
value = model.predict(test_input) # 예측
if(value > 0.5):
	print("긍정적인 리뷰입니다.")
else:
	print("부정적인 리뷰입니다.")


1/1 [==============================] - 0s 24ms/step
긍정적인 리뷰입니다.
